In [1]:
#take tags only from test5 database

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from collections import Counter


#df = pd.read_csv('train.csv')
test_data = pd.read_csv('test2_all.csv')

# delete all data with empty tags from test data
test_data.dropna(subset=['photoTags'], inplace=True)

#take tags from the column 'photoTags'
tags = test_data.loc[:,'photoTags']

# join them to list
text = ' '.join(test_data['photoTags'])
text_list = text.split(' ')

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit_transform(tags)



print(len(vectorizer.vocabulary_))


# save the corpus into the file
import csv

dt = vectorizer.vocabulary_


d = pd.DataFrame.from_dict(dt, orient='index', dtype=None)

d.to_csv("corpus1.csv", encoding="utf-8", header=None)

#LOAD DATA
crp = pd.read_csv("corpus1.csv", header=None)
crp.to_csv("corpus1.csv", encoding="utf-8", header=['words','id'], index=None)

#LOAD DATA
crp = pd.read_csv("corpus1.csv")

#CLEAN DATA
crp['words'] = crp['words'].str.replace("\d+", '')
crp['words'] = crp['words'].str.replace("sec", '')
crp['words'] = crp['words'].str.replace("nabkv", '')
# delete values less then 3
crp =crp[crp['words'].str.len() > 3]
# delete geolat geolon
crp['words'] = crp['words'].str.replace("geolat", '')
crp['words'] = crp['words'].str.replace("geolon", '')
crp['words'] = crp['words'].str.replace("geotagged", '')

#train['words'] = train['words'].str.replace("^[0-9]*$", '')
#s = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", s)
crp['words'].replace('', np.nan, inplace=True)
crp = crp.dropna(subset=['words'])

crp = crp[crp['words'].str.len() > 3]

# save data
crp.to_csv("corpus.csv", encoding="utf-8",index=None)

# load data again 
crp = pd.read_csv('corpus.csv')
#take tags from the column 'photoTags'
tags = crp.loc[:,'words']

# so we have features 
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

tf_matrix = tfidf.fit_transform(tags)

vocab = tfidf.vocabulary_

print(len(vocab))

31180
26908


In [2]:
# we have sparse matrix for 3 mln pictures 
# create KNN
from sklearn.neighbors import NearestNeighbors
train_data = pd.read_csv('dm_clean1.csv')
train_data_ = train_data.loc[:,'photoTags']
_vect = TfidfVectorizer(vocabulary = vocab)
train_c_matrix = _vect.fit_transform(train_data_)

#nbrs = NearestNeighbors(n_neighbors = 1, algorithm='ball_tree')
#nbrs.fit(train_c_matrix)

test_data = pd.read_csv('test2_all.csv')
# delete all data with empty tags from test data
#test_data.dropna(subset=['photoTags'], inplace=True)
test_data['photoTags'].replace( np.nan,'', inplace=True)
test_data_ = test_data.loc[:,'photoTags']
new_vect = TfidfVectorizer(vocabulary = vocab)
test_c_matrix = new_vect.fit_transform(test_data_)


In [3]:
# dimensionality reduction 
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=20, n_iter=7, random_state=42)
svd_test_matrix = svd.fit_transform(test_c_matrix)
svd_train_matrix = svd.fit_transform(train_c_matrix)

In [4]:
nbrs = NearestNeighbors(n_neighbors = 1, algorithm='ball_tree', metric='manhattan', leaf_size=2)
nbrs.fit(svd_train_matrix)

NearestNeighbors(algorithm='ball_tree', leaf_size=2, metric='manhattan',
         metric_params=None, n_jobs=1, n_neighbors=1, p=2, radius=1.0)

In [3]:
nbrs = NearestNeighbors(n_neighbors = 1, algorithm='ball_tree', metric='manhattan', leaf_size=2)
nbrs.fit(train_c_matrix)


C:\Apps\Anaconda3\lib\site-packages\sklearn\neighbors\base.py:211: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "


NearestNeighbors(algorithm='ball_tree', leaf_size=2, metric='manhattan',
         metric_params=None, n_jobs=1, n_neighbors=1, p=2, radius=1.0)

In [5]:
print(1)

1


In [ ]:
# BallTree for faster 
from sklearn.neighbors import KDTree
tree = KDTree( train_c_matrix, leaf_size = 3, metric='manhattan')

In [6]:
import datetime
predicted_lt = []
predicted_lng = []
#indexes = []
print(datetime.datetime.now())

for i in range(0,len(test_data_)):
    distances, indices = nbrs.kneighbors(svd_test_matrix[[i]])
    
    #distances, indices = nbrs.kneighbors(test_c_matrix[i])
    #distances, indices = tree.query([svd_test_matrix[i]], k=1)
    ind = indices[0][0]
    #indexes.append(indices[0][0])
    if i%100==0:
        print(i)
        print(datetime.datetime.now())            
    #print('ind',ind)
    #print(distances)

    lt =train_data.loc[ind,'latitude']
    predicted_lt.append(lt)
    lng = train_data.loc[ind,'longitude']
    predicted_lng.append(lng)
    

    #df_lt = pd.DataFrame([lt], columns=['lt'])
    #df_ln = pd.DataFrame([lng], columns=['ln'])
    
    #data_lt = data_lt.append(df_lt)
    #data_ln = data_ln.append(df_ln)
    #print('test:', test_data.loc[i,'photoTags'])
    #print('train:', train_data.loc[ind,'photoTags'])
    #print('-----------------------------')
    

test_data['lt_predicted'] = predicted_lt
test_data['lng_predicted'] = predicted_lng

test_data.to_csv("predicted_1.csv", encoding="utf-8")

data_predicted = pd.read_csv('predicted_1.csv')

# let's calculate the error and save it in a new column 
lt_error = (data_predicted['lt_predicted'] - data_predicted['latitude'])**(2)
ln_error = (data_predicted['lng_predicted'] - data_predicted['longitude'])**(2)
er = (lt_error + ln_error)**(0.5)
data_predicted['error'] = er

# we will see radius of 0.001 km, 0.01 km, 1 km, 10 km , 100 km , 1000 km, 10000 km, 40000 km 

data_predicted.to_csv("predicted_1_error.csv", encoding="utf-8")

2018-05-16 11:35:48.443557
0
2018-05-16 11:35:48.453558
100
2018-05-16 11:35:50.003634
200
2018-05-16 11:35:51.583684
300
2018-05-16 11:35:53.763732
400
2018-05-16 11:35:55.303783
500
2018-05-16 11:35:57.353873
600
2018-05-16 11:35:58.953902
700
2018-05-16 11:36:00.663983
800
2018-05-16 11:36:02.294035
900
2018-05-16 11:36:04.054070
1000
2018-05-16 11:36:05.524140
1100
2018-05-16 11:36:07.024189
1200
2018-05-16 11:36:08.584241
1300
2018-05-16 11:36:10.214288
1400
2018-05-16 11:36:11.704342
1500
2018-05-16 11:36:13.604402
1600
2018-05-16 11:36:15.264460
1700
2018-05-16 11:36:16.834510
1800
2018-05-16 11:36:18.244532
1900
2018-05-16 11:36:19.774603
2000
2018-05-16 11:36:21.234631
2100
2018-05-16 11:36:23.024689
2200
2018-05-16 11:36:24.474761
2300
2018-05-16 11:36:25.844803
2400
2018-05-16 11:36:27.304848
2500
2018-05-16 11:36:28.624872
2600
2018-05-16 11:36:29.854936
2700
2018-05-16 11:36:31.554992
2800
2018-05-16 11:36:32.995016
2900
2018-05-16 11:36:34.555066
3000
2018-05-16 11:36:36.

In [10]:
print(len(svd_train_matrix[0]))

100


In [7]:
data_predicted = pd.read_csv('predicted_1.csv')

# let's calculate the error and save it in a new column 
lt_error = (data_predicted['lt_predicted'] - data_predicted['latitude'])**(2)
ln_error = (data_predicted['lng_predicted'] - data_predicted['longitude'])**(2)
er = (lt_error + ln_error)**(0.5)
data_predicted['error'] = er

# we will see radius of 0.001 km, 0.01 km, 1 km, 10 km , 100 km , 1000 km, 10000 km, 40000 km 

radius = [0.00001, 0.0001, 0.01, 0.1, 1, 10, 100, 400]

# 1 latitude = 111 km , so we will round it to 100 km
answers = []
for el in radius: 
    answers.append(data_predicted['error'][data_predicted['error']<el].count())
accuracy = []
amount = len(data_predicted)
print(radius)
print(answers)
for i in range(0,8):
    accuracy.append(answers[i]/amount)
print(accuracy)

[1e-05, 0.0001, 0.01, 0.1, 1, 10, 100, 400]
[0, 0, 8, 131, 359, 3797, 13901, 21200]
[0.0, 0.0, 0.00037735849056603772, 0.0061792452830188678, 0.016933962264150944, 0.17910377358490567, 0.65570754716981128, 1.0]


In [11]:
train_c_matrix[0]

<1x143148 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>